정말 좋은 질문이야! **"Jupyter Notebook에서는 왜 `asyncio.run()`을 하면 에러가 날까?"** 이건 파이썬을 공부하는 친구들이 한 번쯤 꼭 겪는 멘붕의 순간이거든.

결론부터 말하자면, **"이미 운전기사(이벤트 루프)가 버스를 운전 중인데, 새로운 운전기사를 또 태우려고 해서"** 그래.

일반 파이썬 파일(`.py`)과 Jupyter Notebook(`.ipynb`)의 결정적인 차이를 아주 쉽게 설명해 줄게.

## 1. 가장 큰 차이: 이벤트 루프의 유무

이전 설명에서 **이벤트 루프**가 일을 시키는 관리자라고 했지? 이 관리자가 언제 출근하는지가 달라.

#### A. 일반 파이썬 환경 (`.py` 파일 실행)

- **상황:** 텅 빈 운동장. 아무도 없어.
  
- **작동:** 네가 `python main.py`를 실행하면, 그제서야 파이썬이 시작돼.
  
- **Asyncio:** 당연히 이벤트 루프(관리자)도 없어. 그래서 네가 `asyncio.run(main())`이라고 명령해야, 그때 관리자가 출근해서 일을 처리하고 퇴근해.
  

#### B. Jupyter Notebook 환경

- **상황:** 이미 축제가 열리고 있는 운동장.
  
- **작동:** Jupyter를 켜는 순간, 백그라운드에서 **IPython 커널**이라는 프로그램이 돌아가.
  
- **Asyncio:** 이 IPython 커널 자체가 이미 **비동기(Async) 방식**으로 돌아가고 있어. 즉, **이미 이벤트 루프(관리자)가 출근해서 24시간 근무 중**인 상태야.
  
  - 왜냐고? 네가 언제 코드 셀을 실행(Shift+Enter)할지 모르니까, 항상 대기하면서 입력을 받아야 하거든.

## 2. `asyncio.run()`을 쓰면 생기는 충돌

자, 이제 문제가 생겨. Jupyter에서는 이미 관리자가 눈을 부릅뜨고 일하고 있는데,  
네가 코드 셀에 이렇게 쓴 거야.

```Python
# Jupyter에서 실행하면 에러 발생!
asyncio.run(main())
```

이건 마치 **이미 버스가 달리고 있는데(루프 실행 중), 시동을 또 켜려고 하는 것**과 같아.  
그래서 파이썬은 이런 에러를 뱉어내.

> **RuntimeError: asyncio.run() cannot be called from a running event loop**
> 
> (해석: 이미 이벤트 루프가 돌아가는 중이라서 `run()`을 또 실행할 수 없어요!)

## 3. 해결 방법: Jupyter만의 특별한 방식

Jupyter Notebook 환경에서는 `asyncio.run()`을 쓸 필요가 없어.  
이미 루프가 돌고 있으니까, 그냥 **할 일을 던져주기만 하면 돼.**

#### 방법 1: `await`를 그냥 바로 쓰기 (Top-level Await)

이게 제일 신기한 거야. 원래 `await`는 `async def` 함수 안에서만 쓸 수 있다고 했잖아?  
하지만 **Jupyter Notebook은 편의를 위해 셀 맨 바깥에서도 `await`를 쓸 수 있게 허용해 줘.**

**[일반 파이썬 vs Jupyter 비교]**

| **일반 파이썬 (.py)** | **Jupyter Notebook (.ipynb)** |
| --- | --- |
| **반드시 `asyncio.run()` 필요** | **`asyncio.run()` 필요 없음** |
| `await`는 함수 안에서만 사용 가능 | `await`를 맨 바깥 줄(Top-level)에서도 사용 가능 |

In [3]:
import asyncio

async def say_hello():
    await asyncio.sleep(1)
    return "Hello, Jupyter!"

# asyncio.run(say_hello())  <-- 이렇게 하면 에러 남!
# 그냥 바로 await를 붙이면 됨
print(await say_hello())

Hello, Jupyter!


#### 방법 2: `nest_asyncio` 라이브러리 (꼭 `run()`을 써야 한다면)

가끔 다른 사람이 짠 코드를 가져왔는데, 그 안에 `asyncio.run()`이 들어있을 수 있잖아?  
코드를 고치기 귀찮을 때 쓰는 마법의 도구가 있어.

`!pip install nest_asyncio`

이 라이브러리는 이미 돌아가는 루프에 '새로운 루프'를 끼워 넣을 수 있게(Nested) 패치해 주는 역할을 해.

In [2]:
import nest_asyncio
nest_asyncio.apply() # "관리자 안에 또 관리자를 둬도 봐주세요" 라고 허락받는 코드

async def main():
    print("이제 에러 안 남!")

import asyncio
asyncio.run(main()) # 이제 에러 없이 실행됨!

이제 에러 안 남!


고등학생 친구를 위해 요약하자면:

1. **일반 파이썬:** 관리자가 없어서 `asyncio.run()`으로 불러와야 함.
2. **Jupyter:** 이미 관리자가 상주 중임. `asyncio.run()`을 부르면 관리자끼리 싸움(에러) 남.
3. **해결:** Jupyter에서는 `run()` 빼고 그냥 **`await 함수이름()`**만 쓰면 됨.
  

#### 📄 더 자세히 읽어볼 만한 문서 (영어지만 핵심은 이거야)

이 내용이 공식적으로 정리된 문서를 찾고 싶다면 아래 키워드로 검색해 보면 좋아.

1. **IPython 공식 문서 (Autoawait):**
  
  - IPython(Jupyter의 핵심 엔진)이 어떻게 비동기 코드를 처리하는지 설명해.
    
  - 키워드: *IPython autoawait*
    
2. **nest_asyncio 깃허브:**
  
  - 왜 루프 중첩이 안 되는지, 이 라이브러리가 어떻게 해결하는지 나와 있어.
    
  - 키워드: *github nest_asyncio*
    

이제 Jupyter Notebook에서 왜 에러가 났는지, 어떻게 해결하면 되는지 완벽하게 이해됐지? 궁금한 게 또 생기면 언제든 물어봐!